In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [ ]:
list_url = []

for page in range(1,10000, 50):
  url = f'https://www.imdb.com/search/title/?title_type=feature&adult=include&sort=boxoffice_gross_us,desc&start={page}'
  req = requests.get(url)
  soup = BeautifulSoup(req.content, 'html.parser')
  data = soup.find_all('div', {'class':'lister-item mode-advanced'})

  for film in data:     
    title = film.find('h3', {'class':'lister-item-header'}).find('a').text
    link_url = film.find('h3', {'class':'lister-item-header'}).find('a').attrs['href']
    url2 = 'https://www.imdb.com'+link_url
    list_url.append(url2)

In [ ]:
data = {}

for url in list_url:
  try:
    res = requests.get(url, timeout=20)
  except:
    pass
  soup = BeautifulSoup(res.content,'lxml')
  scripts = soup.find_all('script')
  scripts = scripts[-1]
  json_data = json.loads(scripts.string)
  del json_data['query']
  del json_data['runtimeConfig']
  del json_data['dynamicIds']
  del json_data['props']['pageProps']['requestContext']
  del json_data['props']['pageProps']['cmsContext']
  del json_data['props']['pageProps']['translationContext']
  data[url] = json_data

with open('imdb.txt', 'w') as convert_file:
     convert_file.write(json.dumps(data))